# Часть 1: Общие задачи - EDA СберАвтоподписка

**Бизнес-контекст:** Сервис "СберАвтоподписка" - долгосрочная аренда авто.
**Цель:** Подготовить данные для ML-модели предсказания конверсии.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette('husl')
print('✅ Библиотеки загружены')

## 1.1 Загрузка данных

In [ ]:
def load_data(filename, path='../data/'):
    pkl_path = os.path.join(path, filename + '.pkl')
    csv_path = os.path.join(path, filename + '.csv')
    if os.path.exists(pkl_path):
        return pd.read_pickle(pkl_path)
    elif os.path.exists(csv_path):
        return pd.read_csv(csv_path)
    raise FileNotFoundError(f'{filename} не найден')

ga_sessions = load_data('ga_sessions')
ga_hits = load_data('ga_hits')
print(f'Sessions: {ga_sessions.shape}')
print(f'Hits: {ga_hits.shape}')

In [ ]:
# Информация о датасетах
print('GA Sessions:')
print(ga_sessions.info())
print('\nMissing values (%):')
print((ga_sessions.isnull().sum() / len(ga_sessions) * 100).round(2))

## 1.2 Очистка данных

In [ ]:
sessions_clean = ga_sessions.copy()
hits_clean = ga_hits.copy()

# Удаление дубликатов
sessions_clean = sessions_clean.drop_duplicates()
hits_clean = hits_clean.drop_duplicates()

# Типизация
for col in ['visit_date', 'hit_date']:
    if col in sessions_clean.columns:
        sessions_clean[col] = pd.to_datetime(sessions_clean[col])
    if col in hits_clean.columns:
        hits_clean[col] = pd.to_datetime(hits_clean[col])

cat_cols = ['utm_source', 'utm_medium', 'device_category', 'device_os', 'geo_country', 'geo_city']
for col in cat_cols:
    if col in sessions_clean.columns:
        sessions_clean[col] = sessions_clean[col].astype('category')

print('✅ Данные очищены')

## 1.3 Визуализации

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

if 'utm_source' in sessions_clean.columns:
    sessions_clean['utm_source'].value_counts().head(10).plot(kind='barh', ax=axes[0,0])
    axes[0,0].set_title('Топ-10 каналов')

if 'device_category' in sessions_clean.columns:
    sessions_clean['device_category'].value_counts().plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%')
    axes[0,1].set_title('Типы устройств')
    axes[0,1].set_ylabel('')

if 'geo_country' in sessions_clean.columns:
    sessions_clean['geo_country'].value_counts().head(10).plot(kind='barh', ax=axes[1,0])
    axes[1,0].set_title('Топ-10 стран')

if 'visit_date' in sessions_clean.columns:
    sessions_clean.groupby('visit_date').size().plot(ax=axes[1,1])
    axes[1,1].set_title('Динамика визитов')

plt.tight_layout()
plt.savefig('../data/eda_plots.png')
plt.show()

## 1.4 Корреляционный анализ

In [ ]:
numeric_cols = sessions_clean.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 1:
    corr = sessions_clean[numeric_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
    plt.title('Корреляционная матрица')
    plt.savefig('../data/correlation.png')
    plt.show()

## 1.5 Соединение таблиц

In [ ]:
hits_agg = hits_clean.groupby('session_id').agg({
    'hit_number': 'count'
}).reset_index()
hits_agg.columns = ['session_id', 'total_hits']

merged = sessions_clean.merge(hits_agg, on='session_id', how='left')
print(f'Объединенная таблица: {merged.shape}')
merged.to_pickle('../data/merged_data.pkl')
print('✅ Данные сохранены')